# Scatter in a Scatter Test --- Bottom Line: Very Messy Output

## each scatter block creates a directory tree in each tree:


### (terminal w docker) Start the container:
```bash
docker run -p 8888:8888 -v `pwd`:/home/tmpusr/run_dir -it aka1spanky/ncsa:devwdlV01 jupyter notebook --port=8888 --ip=0.0.0.0 --allow-root
```
### (python3) In notebook -- running in the container -- path to cromwell, path to run_directory:
```python
os.listdir('/usr/locl/etc/')
os.listdir('/home/tmpusr/run_dir/'
```

### (terminal w docker) Run string example:
```bash
docker exec -i funny_docker java -jar /usr/locl/etc/cromwell-36.jar /home/tmpusr/run_dir/run scatter_hello.wdl
```


In [1]:
import os
import sys
import numpy as np
import pandas as pd

In [4]:
# running jupyter inside the container - this is the externally mounted directory
os.getcwd()

'/home/tmpusr/run_dir'

In [5]:
#           location of the cromwell files
os.listdir('/usr/locl/etc')

['womtool-36.jar', 'cromwell-36.jar', '.DS_Store']

In [10]:
#           before and after cell >> cromwell-* will show up after running the next cell
#           Iff and Only If you run the writefile below 
os.listdir()

['scatter_hello.wdl',
 'DC_Run_scatter_sqr.ipynb',
 '.ipynb_checkpoints',
 'Input_Array_Indexing_Examples.ipynb']

In [47]:
%%writefile scatter_hello.wdl
###########################
# Usage: 
# java -jar full/path/to/cromwell run scatter_hello.wdl
###########################

task hello {
    String addressee_1
    String addressee_2
    Int ix_1
    Int ix_2
    
    command {
      echo "[${ix_1}][${ix_2}][Left]: ${addressee_1}\n[${ix_1}][${ix_2}][Right]: ${addressee_2}"
    }
    
    output {
        String message = read_string(stdout())
    }
}

Overwriting scatter_hello.wdl


In [48]:
%%writefile call_scatter_hello.wdl
###########################
#           call_scatter_hello.wdl
###########################

import "/home/tmpusr/run_dir/scatter_hello.wdl" as HELLO

workflow inner_hello {
    Array[Array[String]] InnerInputStrArr
    Int ix_1
    
    Int arr_inp_size = length(InnerInputStrArr)
    Array[Int] InnerIndex = range(arr_inp_size)
    
    scatter (idx in InnerIndex) {
        
        call HELLO.hello as howdy {
            input:
                addressee_1 = InnerInputStrArr[idx][0],
                addressee_2 = InnerInputStrArr[idx][1],
                ix_1 = ix_1,
                ix_2 = idx
        }
    }
    
#     output {
#         Array[String] message = howdy.message
#     }
}

Overwriting call_scatter_hello.wdl


In [49]:
%%writefile scatter_call_scatter_hello.wdl
###########################
#           scatter_call_scatter_hello.wdl
###########################

import "/home/tmpusr/run_dir/call_scatter_hello.wdl" as CALL_SCATTER

workflow outer_hello {
    Array[Array[Array[String]]] InStrArr = [ [ ["0:0:0", "0:0:1"], ["0:1:0","0:1:1"] ],  [ ["1:0:0", "1:0:1"], ["1:1:0","1:1:1"] ] ]
    Int arr1_size = length(InStrArr)
    Array[Int] Index = range(arr1_size)
    
    scatter (idx in Index) {
        
        call CALL_SCATTER.inner_hello as hiho {
            input:
                InnerInputStrArr = InStrArr[idx],
                ix_1 = idx
        }
    }
    
    output {
        Array[Array[String]] hiho_out = hiho.message
    }
}

Overwriting scatter_call_scatter_hello.wdl


### Note that directory names will differ after container is closed.

In [15]:
#           before and after cell >> cromwell-* will show up after running the next cell
#           Iff and Only If you run the writefile below 
base_dir = '/home/tmpusr/run_dir/cromwell-executions'
dir_string = os.path.join(base_dir, 'outer_hello/73bca2ff-6f0c-4b85-94cd-f8ac8e7518db')
inner_shard = os.path.join(dir_string, 'call-hiho', 'shard-0', 'CALL_SCATTER.inner_hello')
new_tree = os.path.join(inner_shard, '73188a57-669d-4007-b05a-d4c94f5dbe3c')
os.listdir(new_tree)

['call-howdy']

In [18]:
dir_string_II = os.path.join(new_tree, 'call-howdy', 'shard-0', 'execution')
os.listdir(dir_string_II)

['stdout',
 'script.background',
 'stderr',
 'script.submit',
 'script',
 'stdout.background',
 'rc',
 'stderr.background']

In [21]:
out_file = os.path.join(dir_string_II, 'stdout')
with open(out_file, 'r') as fh:
    lines = fh.readlines()
    
for line in lines:
    print(line)

[0][0][Left]: 0:0:0\n[0][0][Right]: 0:0:1



In [22]:
def get_stdout_wdl_list(base_dir):
    """ Usage: outies = get_stdout_wdl_list(base_dir) """
    look_4_file = 'stdout'
    stdout_wdl_list = []
    for dir_name, dirs_list, files_list in os.walk(base_dir):
        for file_name in files_list:
            if len(file_name) == len(look_4_file) and file_name == look_4_file:
                stdout_wdl_list.append(os.path.join(dir_name, file_name))
    return stdout_wdl_list

outies = get_stdout_wdl_list(base_dir=dir_string)
for outie in outies:
    with open(outie, 'r') as fh:
        lines = fh.readlines()
    for line in lines:
        print(line)

[0][0][Left]: 0:0:0\n[0][0][Right]: 0:0:1

[0][1][Left]: 0:1:0\n[0][1][Right]: 0:1:1

[1][0][Left]: 1:0:0\n[1][0][Right]: 1:0:1

[1][1][Left]: 1:1:0\n[1][1][Right]: 1:1:1

